In [ ]:
# Module imports

import os
from os import path

import matplotlib.pyplot as plt

from ase.io import read, write
from ase.optimize import QuasiNewton, BFGS, LBFGS, FIRE
from ase.calculators.vasp import Vasp
from ase.vibrations import Vibrations
from ase.units import Bohr, Rydberg, kJ, kB, fs, Hartree, mol, kcal
from ase.constraints import FixAtoms
from ase.neb import NEB, NEBTools
from ase.dyneb import DyNEB

from xtb.ase.calculator import XTB


In [ ]:
def opt_xtb_clust(atoms, save=False, basedir=None):
    """
    Optimize the geometry of an atomic cluster using XTB and QuasiNewton optimization.
    
    Parameters:
        atoms (ase.Atoms): The atomic configuration.
        save (bool): Whether to save the optimized structure.
        basedir (str): Directory where the optimized structure is saved.
        
    Returns:
        atoms (ase.Atoms): The optimized atomic configuration.
    """
    # Set up XTB calculator with GFN1-xTB method
    atoms.calc = XTB(method="GFN1-xTB", electronic_temperature=300.0, accuracy=1.0)
    
    # Optimize geometry using QuasiNewton algorithm
    optimizer = QuasiNewton(atoms)
    optimizer.run(fmax=0.01, steps=300)
    
    # Save the optimized structure if requested
    if save and basedir is not None:
        target_dir = os.path.join(basedir, str(atoms.symbols))
        try:
            os.mkdir(target_dir)
        except Exception:
            print(f"{target_dir}: folder exists!")
        write(os.path.join(target_dir, 'opt.xyz'), atoms)
    
    return atoms


In [ ]:
# Helper functions for saving NEB images and figures
def save_neb_images(images, folder, basedir):
    """
    Save NEB images to disk.
    """
    folder_path = os.path.join(basedir, folder)
    try:
        os.mkdir(folder_path)
    except Exception:
        print(f"{folder_path} folder exists! Skipping folder creation.")
    for idx, image in enumerate(images, start=1):
        write(os.path.join(folder_path, f'neb{idx}.xyz'), image)

def save_neb_figures(images, folder, basedir):
    """
    Generate and save the NEB band plot evolution.
    """
    folder_path = os.path.join(basedir, folder)
    nebtools = NEBTools(images)
    # Calculate barrier and maximum force (not used further here)
    Ef, dE = nebtools.get_barrier(fit=False)
    max_force = nebtools.get_fmax()
    # Save the primary NEB band plot
    fig = nebtools.plot_band()
    fig.savefig(os.path.join(folder_path, 'NEB.png'))


## Optimization of initial and final states for a given elementary step + vibrational frequency analysis

In [ ]:
# Read the initial structure from file to start optimization of an initial or final state (pre- or post-TS).
atoms = read('init.xyz')


In [ ]:
basedir = '/Users/user/calcs/calc'

In [ ]:
# Note: Ensure that 'basedir' is defined (e.g., in a previous cell or set it here)
opt_xtb_clust(atoms, save=True, basedir=basedir)


In [ ]:
# Define the number of metal atoms (assumed to appear before adsorbates)
n_metal_atoms = 79

# Perform vibrational analysis on the adsorbate atoms only
adsorbate_indices = list(range(n_metal_atoms, len(atoms)))
vib = Vibrations(atoms, adsorbate_indices, delta=0.01, nfree=2)
vib.run()


In [ ]:
# Display the vibrational frequency summary and write output for Jmol visualization
vib.summary()
vib.write_jmol()


## Optimized (Dy/CI-)NEB Protocol

In [ ]:
# Note: DyNEB parameters are tightened consecutively in the following protocols,
# representing increasingly strict convergence criteria.

# NEB parameters and base directory definition
n_images_par = 7  # Number of intermediate images

# Define initial and final states
initial = read('reac.xyz')
final = read('prod.xyz')

# Create NEB band with linearly interpolated images
images = [initial] + [initial.copy() for _ in range(n_images_par)] + [final]

# Set XTB calculator for each image
for image in images:
    image.calc = XTB(method="GFN1-xTB", electronic_temperature=300.0, accuracy=1.0)

# ----------------------
# NEB Protocol 1
neb = DyNEB(
    images,
    k=0.1,
    fmax=0.1,
    climb=False,
    remove_rotation_and_translation=False,
    dynamic_relaxation=True,
    method='string',
    scale_fmax=1.0
)
# Use IDPP interpolation (fallback to linear interpolation if necessary)
neb.interpolate('idpp', apply_constraint=False)

optimizer = LBFGS(neb)  # Removed trajectory saving
optimizer.run(fmax=1.0, steps=100)

save_neb_images(images, 'NEB_1', basedir)
save_neb_figures(images, 'NEB_1', basedir)

# ----------------------
# NEB Protocol 2
fmax_par = 0.4
neb_ci = DyNEB(
    images,
    k=0.1,
    fmax=0.01,
    climb=True,
    remove_rotation_and_translation=False,
    dynamic_relaxation=True,
    method='aseneb',
    scale_fmax=1.0
)
optimizer = FIRE(neb_ci, maxstep=0.05)
optimizer.run(fmax=fmax_par, steps=100)

save_neb_images(images, 'NEB_2', basedir)
save_neb_figures(images, 'NEB_2', basedir)

# ----------------------
# NEB Protocol 3 (Reduced maxstep)
fmax_par = 0.2
neb_ci = DyNEB(
    images,
    k=0.1,
    fmax=0.01,
    climb=True,
    remove_rotation_and_translation=False,
    dynamic_relaxation=True,
    method='aseneb',
    scale_fmax=1.0
)
optimizer = FIRE(neb_ci, maxstep=0.02)
optimizer.run(fmax=fmax_par, steps=100)

save_neb_images(images, 'NEB_3', basedir)
save_neb_figures(images, 'NEB_3', basedir)

# ----------------------
# NEB Protocol 4
fmax_par = 0.1
neb_ci = DyNEB(
    images,
    k=0.1,
    fmax=0.01,
    climb=True,
    remove_rotation_and_translation=False,
    dynamic_relaxation=True,
    method='aseneb',
    scale_fmax=1.0
)
optimizer = FIRE(neb_ci, maxstep=0.01)
optimizer.run(fmax=fmax_par, steps=100)

save_neb_images(images, 'NEB_4', basedir)
save_neb_figures(images, 'NEB_4', basedir)

# ----------------------
# NEB Protocol 5
fmax_par = 0.03
neb_ci = DyNEB(
    images,
    k=0.1,
    fmax=0.001,
    climb=True,
    remove_rotation_and_translation=False,
    dynamic_relaxation=True,
    method='aseneb',
    scale_fmax=1.0
)
# Increse maxstep if the convergence is stable. maxstep=0.005 is preselected for difficult cases.
optimizer = FIRE(neb_ci, maxstep=0.005)
optimizer.run(fmax=fmax_par, steps=100)

save_neb_images(images, 'NEB_5', basedir)
save_neb_figures(images, 'NEB_5', basedir)


## Wrapping up: final NEB plot and vibrational frequency analysis

In [ ]:
nebtools = NEBTools(images)

# Retrieve barrier information without interpolation
Ef, dE = nebtools.get_barrier(fit=False)
max_force = nebtools.get_fmax()

# Generate and save the overall NEB plot
fig = nebtools.plot_band()
fig.savefig('NEB.png')


In [ ]:
# Perform vibrational frequency analysis on the optimized transition state
# (Only the adsorbate atoms are considered; metal atoms are defined first
# in the atoms object, just as in the initial XYZ files)

# Select the TS image using the NEB plot /PES profile
ts_image_number = 6
ts = images[ts_image_number - 1]

n_metal_atoms = 79
adsorbate_indices = list(range(n_metal_atoms, len(atoms)))
vib = Vibrations(ts, adsorbate_indices, delta=0.01, nfree=2)
vib.run()


In [ ]:
# Display a summary of the vibrational frequency analysis,
# which includes the computed frequencies and any associated imaginary modes.
vib.summary()

# Write output files in a format suitable for Jmol visualization,
# allowing for further inspection of the vibrational modes.
vib.write_jmol()